# Searchlight Analysis
Searchlight Analysis is used to sequentially analyze small groups of voxels in the brain in order to identify regions of interest. The technique is designed in the following steps:

1. Define a sphere of voxels around a seed voxel
2. Extract the time series from each voxel in the sphere
3. Concatenate the time series into a feature vector
4. Train a classifier on the feature vector and the labels
5. Use the classifier to predict the labels of the seed voxel
6. Repeat steps 1-5 for all seed voxels
7. Aggregate the results across all seed voxels to form a statistical map


In [ ]:
# Import libraries
